<a href="https://colab.research.google.com/github/DelmiroDaladier/NLP-studies/blob/master/text_generation(Pytorch)/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
!pip install unidecode

In [92]:
import unidecode
import string
import random
import re

import nltk
from nltk.corpus import machado

import torch
import torch.nn as nn
from torch.autograd import Variable

In [93]:
nltk.download('machado')

[nltk_data] Downloading package machado to /root/nltk_data...
[nltk_data]   Package machado is already up-to-date!


True

In [94]:
raw_text = machado.raw('contos/macn001.txt')
raw_text = raw_text.split('\n\nCAPÍTULO PRIMEIRO\n\n')[1]
raw_text

"Era conveniente ao romance que o leitor\nficasse muito tempo sem saber quem era Miss Dollar. Mas por outro lado,\nsem a apresentação de Miss Dollar, seria o autor obrigado a longas\ndigressões, que encheriam o papel sem adiantar a ação. Não há hesitação\npossível: vou apresentar-lhes Miss Dollar.\n\nSe o leitor é rapaz e dado ao gênio\nmelancólico, imagina que Miss Dollar é uma inglesa pálida e delgada,\nescassa de carnes e de sangue, abrindo à flor do rosto dois grandes olhos azuis\ne sacudindo ao vento umas longas tranças loiras. A moça em questão deve ser\nvaporosa e ideal como uma criação de Shakespeare; deve ser o contraste do roastbeef\nbritânico, com que se alimenta a liberdade do Reino Unido. Uma tal Miss\nDollar deve ter o poeta Tennyson de cor e ler Lamartine no original; se\nsouber o português deve deliciar-se com a leitura dos sonetos de Camões ou os Cantos\nde Gonçalves Dias. O chá e o leite devem ser a alimentação de semelhante\ncriatura, adicionando-se-lhe alguns confei

In [95]:
character_set = string.printable
character_to_index = {}
n_characters = len(character_set)

for character in character_set:
  if character not in character_to_index:
    character_to_index[character] = len(character_to_index)

In [96]:
raw_text = unidecode.unidecode(raw_text)
raw_text = raw_text.replace('\n', ' ')
len_text = len(raw_text)
len(f'{raw_text}')

47198

In [107]:
chunk_len = 200

def random_text_chunk():
    start = random.randint(0, len_text - chunk_len)
    end = start+ chunk_len + 1
    return raw_text[start: end]

def string_to_tensor(sentence, character_to_index):
  tensor = torch.zeros(len(sentence)).long()
  for char in range(len(sentence)):
    tensor[char] = character_to_index[sentence[char]]
  return Variable(tensor)

def create_training_set():
  chunk = random_chunk()
  input = string_to_tensor(chunk[:-1], character_to_index)
  target = string_to_tensor(chunk[1:], character_to_index)
  return input, target

In [108]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, n_layers=1):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.n_layers = n_layers

    self.encoder = nn.Embedding(input_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
    self.decoder = nn.Linear(hidden_size, output_size)

  def forward(self, input, hidden):
    input = self.encoder(input.view(1, -1))
    output, hidden = self.gru(input.view(1, 1, -1), hidden)
    output = self.decoder(output.view(1, -1))
    return output, hidden

  def init_hidden(self):
    return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [109]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
  hidden = decoder.init_hidden()
  prime_input = string_to_tensor(prime_str, character_to_index)
  predicted = prime_str

  for p in range(len(prime_str) - 1):
    _, hidden = decoder(prime_input[p], hidden)
  inp = prime_input[-1]

  for p in range(predict_len):
    output, hidden = decoder(inp, hidden)

    output_dist = output.data.view(-1).div(temperature).exp()
    top_i = torch.multinomial(output_dist, 1)[0]

    predicted_char = character_set[top_i]
    predicted += predicted_char
    inp = string_to_tensor(predicted_char, character_to_index)

  return predicted

In [110]:
import time, math

def time_since(since):
  s = time.time() - since
  m = math.floor(s / 60)
  s -= m * 60
  return '%dm %ds' % (m, s)

In [111]:
def train(input, target):
  hidden = decoder.init_hidden()
  decoder.zero_grad()
  loss = 0
  for c in range(chunk_len):
    output, hidden = decoder(input[c], hidden)
    loss += criterion(output, target[c].unsqueeze(0))

  loss.backward()
  decoder_optimizer.step()

  return loss.item() / chunk_len

In [112]:
n_epochs = 2000
print_every = 100
plot_every = 10
hidden_size = 100
n_layers = 1
lr = 0.005

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
  loss = train(*create_traning_set())
  loss_avg += loss

  if epoch % print_every == 0:
    print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
    print(evaluate('Wh', 100), '\n')

  if epoch % plot_every == 0:
    all_losses.append(loss_avg / plot_every)
    loss_avg = 0

 15s (100 5%) 2.1025]
Wha da  dia de o elho os seitos se o miu ha ecido um; que que de porindontantontou.  E renda astide po 

 31s (200 10%) 1.9930]
Whuentara estar mentor o e sois.  A velega. Codo e o pessapeno oue ter unda. Erera as entorde indo lon 

 47s (300 15%) 1.9516]
Whe-as teva mim ainde Nar o cardas. E cama cara ou a fentusidate, persou-se de plos ser do couturr da  

 3s (400 20%) 1.8045]
Wh! estrida o que cormenia proncamento mesto dio gualvar incampo, que Mendrade do abelo Ras les de a l 

 18s (500 25%) 1.8698]
Whe; lichado aparecse um ventente a ao com um mem com para que escunte as que Mendonca a jas leitos co 

 34s (600 30%) 1.8109]
Whe se conhavel quar ferito; a comulhado a rapaos se que senhora responca prombre a megor lidou-lhe do 

 50s (700 35%) 1.5252]
Wh! luveta-lhe amargui aboisa e mele ela casa com a mas e riz do munuvava acasa, a com a gratiu ela da 

 5s (800 40%) 1.5570]
Wh$us parecerdo. Era a pedeu despedir-sista osma na sa e a com cara parrida e algomo

In [116]:
print(evaluate('A', 200, temperature=0.5))

A-cava as com o motivo e dia do sentar-se pelo na senhor a carta a Miss Dollar e com o com o mende e a corta a corando de outra de olhos de Margarida com Mendonca a casa de conse alguma nao latio de Ma
